In [1]:
import pandas as pd
import numpy as np
from typing import Dict

from PIL import Image
from datasets import load_dataset, Dataset, Features, Value

from sklearn.model_selection import train_test_split

# Data

In [5]:
# df2 = pd.read_json('../data/vlm.jsonl', lines=True)
df2 = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=1000, random_state=1)
# df2 = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=3000, random_state=1)


categories = list(filter(lambda x: x != None, pd.DataFrame([i for i in list(df2['annotations'].apply(lambda x: [i['caption'] for i in x]))]).to_numpy().flatten()))
num_cat = len(categories)
categories = pd.DataFrame(categories)
categories.drop_duplicates(inplace=True)
categories = categories.values.flatten().tolist()
id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

df2["image_id"] = df2.index

df2['width'] = df2['image'].apply(lambda x: Image.open('../data/images/' + x).size)
df2['height'] = df2['width'].apply(lambda x: x[1])
df2['width'] = df2['width'].apply(lambda x: x[0])

df2['img'] = df2['image'].apply(lambda x: Image.open('../data/images/' +x))

df2['area'] = df2['annotations'].apply(lambda x: {"area":[i['bbox'][2] * i['bbox'][3] for i in x]})
df2['bbox'] = df2['annotations'].apply(lambda x: {"bbox":[i['bbox'] for i in x]})
df2['category'] = df2['annotations'].apply(lambda x: {'category':[label2id[i['caption']] for i in x]})

df2 = df2.drop(columns=['annotations'])

counter = 0
lst = []
for i in df2['category']:
	size = len(i['category'])
	each = list(np.arange(counter, counter + size))
	lst.append({'id':each})
	counter += size
df2['id'] = lst

df_vlm = pd.DataFrame({
	"file_name" : df2['image'],
	"image_id" : df2['image_id'],
	"image" : df2['img'],
	"width" : df2['width'],
	"height": df2['height'],
	"objects": df2.apply(lambda x: {'id': x['id']['id'], 'area': x['area']['area'],'bbox': x['bbox']['bbox'], 'category': x['category']['category']}, axis=1)
})
# df_vlm.to_csv('meta_data.csv', index=False)
df_vlm

,file_name,image_id,image,width,height,objects
5074,image_5074.jpg,5074,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [0, 1, 2, 3, 4, 5, 6, 7], 'area': [2640..."
2711,image_2711.jpg,2711,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [8, 9, 10, 11, 12], 'area': [2048, 2496..."
3670,image_3670.jpg,3670,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [13, 14, 15, 16], 'area': [960, 3072, 1..."
2377,image_2377.jpg,2377,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [17, 18, 19, 20, 21, 22, 23], 'area': [..."
4195,image_4195.jpg,4195,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [24, 25, 26, 27, 28, 29, 30], 'area': [..."
...,...,...,...,...,...,...
1357,image_1357.jpg,1357,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16302, 16303, 16304, 16305, 16306, 163..."
3892,image_3892.jpg,3892,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16308, 16309, 16310, 16311, 16312, 163..."
3258,image_3258.jpg,3258,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16314, 16315, 16316, 16317, 16318], 'a..."
3507,image_3507.jpg,3507,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16319, 16320, 16321], 'area': [1536, 1..."


# Split Dataset

In [4]:
df_vlm_csv = df_vlm.drop(columns=['image'])
df_vlm_csv.to_csv('../vlm-images2/images/metadata.csv', index=False)
pd.DataFrame({
	'name':df_vlm['file_name'],
	'img':df_vlm['image']
}).apply(lambda x: x[1].save('../vlm-images2/images/' + x[0]), axis=1)

C:\Users\Keck\AppData\Local\Temp\ipykernel_23788\4127860644.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  }).apply(lambda x: x[1].save('../vlm-images2/images/' + x[0]), axis=1)


5074    None
2711    None
3670    None
2377    None
4195    None
        ... 
1357    None
3892    None
3258    None
3507    None
1978    None
Length: 3000, dtype: object